# Initial dataset exploration

This notebook is meant to give an initial insight into the dataset. We load the data and perform an initial explorative data analysis procedure. 

Conclusions:
- Depending on the columns used for the model, we need to find a suitable way to impute data, because there are quite some missing values.
- There is a lot of categorical data with high number of categories. So this needs to be dealt with appropriately. Either by means of dimensionality reduction, or by means of proper feature selection. 

In [2]:
import pandas as pd

# Load the Excel file
file_path = '../data/luiss_data_anonym.xlsx'
df = pd.read_excel(file_path)

Dataset columns are extensively described in the "Bip xTech and TeamSystem - Dataset Description.pdf" file so we don't see the reason to re-describe all the columns

In [3]:
# First we display the shape of the dataset (rows, columns)
print("Dataset Shape:", df.shape)

# Then we look at the different data types of each column
print("\nData Types:\n", df.dtypes)

Dataset Shape: (134437, 45)

Data Types:
 Unnamed: 0                  int64
A                          object
Ateco                       int64
B                          object
C                         float64
DataDoc            datetime64[ns]
D                          object
E                          object
F                          object
G                          object
H                          object
Tdoc                       object
VA                         object
CE                         object
DescrizioneRiga            object
Importo                   float64
Iva                        object
Conto                       int64
ContoStd                    int64
IvaM                      float64
Comp                      float64
Iva11                     float64
TM                          int64
%Forf                     float64
Art1                       object
Valore1                    object
Nomenclatura               object
Ritac                     float64
RF    

Output shows the datatypes of each column

In [4]:
# number of missing values in IvaM (TARGET) column:
print(f"{len(df[df.IvaM.isna()].index)} / {len(df.index)}")

471 / 134437


Output shows how many rows of the total number of rows have IvaM column missing

In [5]:
# First of all, delete all rows for which IvaM is missing, because we need to predict this:
df = df[~df.IvaM.isna()]

# Calculate number of NaNs per column
nans_per_column = df.isnull().sum()

# Calculate percentage of NaNs per column
percentage_nans_per_column = (df.isnull().sum() / len(df)) * 100

# Combine both into a DataFrame for a cleaner display
nans_df = pd.DataFrame({'Number of NaNs': nans_per_column, 'Percentage of NaNs': percentage_nans_per_column})

print(nans_df)

                 Number of NaNs  Percentage of NaNs
Unnamed: 0                    0            0.000000
A                             0            0.000000
Ateco                         0            0.000000
B                            54            0.040309
C                        125230           93.478942
DataDoc                       0            0.000000
D                             1            0.000746
E                        132175           98.663094
F                        132175           98.663094
G                        133948           99.986564
H                        133948           99.986564
Tdoc                          0            0.000000
VA                            0            0.000000
CE                       131878           98.441396
DescrizioneRiga             905            0.675545
Importo                       0            0.000000
Iva                           0            0.000000
Conto                         0            0.000000
ContoStd    

Output shows number of and percentage of NaN values per column

**We now can identify three groups of columns:**

1. Columns with no missing values
2. Columns with a small percentage of missing values (<5%)
3. Columns with a large percentage of missing values (>50%)

_Note no cases between 2. and 3._

Before deleting rows or columns we have to ask two questions:

- does the presence of a values in a column with many NaNs provide a lot of predictive power? This way we could use empty values
- is the column with few NaNs valuable enough to apply a data imputation technique?

In [6]:
# Analyze distributions of all numerical columns
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Unnamed: 0,133966.0,92898.817528,1.0,33609.25,67119.5,200972.75,234501.0,77400.095622
Ateco,133966.0,471199.051715,11110.0,432101.0,471920.0,479110.0,960909.0,185118.288306
C,8736.0,85.082074,1.0,100.0,100.0,100.0,100.0,31.191071
DataDoc,133966,2022-06-12 18:10:18.174760960,2020-01-09 00:00:00,2021-09-30 00:00:00,2022-08-31 00:00:00,2023-03-27 00:00:00,2023-09-29 00:00:00,NaN
Importo,133966.0,831.635312,-459751.5,10.13,37.8,220.4,4000000.0,17873.158982
Conto,133966.0,5789692.132392,60010.0,5805125.0,5810005.0,6625015.0,9999009.0,1437093.517121
ContoStd,133966.0,5789569.353358,60010.0,5805125.0,5810005.0,6625005.0,9515090.0,1437093.036622
IvaM,133966.0,318.052707,2.0,312.0,324.0,374.0,722.0,77.258443
Comp,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Iva11,33734.0,11.015741,1.0,9.0,9.0,9.0,35.0,6.45868


In [9]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

# count unique values (# categories) in each categorical column
unique_counts = df[categorical_columns].nunique()
unique_counts

A                     13
B                      5
D                      3
E                      4
F                      6
G                      2
H                      2
Tdoc                  11
VA                     2
CE                     4
DescrizioneRiga    62321
Iva                   24
Art1                 474
Valore1            43984
Nomenclatura           1
RF                    12
RifNormativo         647
CMar                   4
CTra                   2
Rev                    2
CVia                   2
Rifamm               155
X                      3
Art2                  58
Valore2             1072
Art3                  21
Valore3              324
dtype: int64

Because of the high number of classes in the target value it is difficult to plot correlations in an intuitive way. Rather the first thing we decided to do is train a network based on all categories, to find out what we could learn from this. 